In [1]:
import pandas as pd
import os
import shutil #for 2C
from openpyxl import load_workbook #for 2C
import statistics as stats
from scipy.stats import mode

In [2]:
os.chdir('C:\\Users\\DavidEttinger02\\Desktop\\Programming\\main')
os.getcwd()
os.listdir('.')

['20160914-tptversion4.xls',
 'edited.xlsx',
 'Programming Exercise.pdf',
 'Programming Exercise.rar',
 'Python Installation Guide.pdf',
 'Sample TPT V3 Report.xlsx']

In [5]:
excel_file = 'Sample TPT V3 Report.xlsx'

assignment = pd.read_excel(excel_file)
sheet = pd.read_excel(excel_file, sheet_name=0)

sheet.head()
shape = sheet.shape
totalrows = shape[0]

In [6]:
def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + t           

    avg = sum_num / len(num)
    return avg

#Task 1 A
totalnetassets = sheet['28_Position-Valuation-MarketExposurePC'].sum()
shareprice = sheet['8_Portfolio-ShareClass-SharePrice'].mean()
numberofshares = totalnetassets/shareprice

print('The total worth of the assets is:', totalnetassets)
print('The total number of shares:', numberofshares)

#Task 1 B
currencycolumn = sheet['21_Position-Valuation-QuotationCurrency']
#create lists of currency rows for USD and CHF
cellswithchf = 0
cellswithusd = 0
rowswithchf = []
rowswithusd = []
track = 0
for x in currencycolumn:
    track = track + 1
    if x == 'CHF':
        cellswithchf = cellswithchf + 1
        rowswithchf.append(track+1)
    elif x == 'USD':
        cellswithusd = cellswithusd + 1
        rowswithusd.append(track+1)        
#create lists of currency rows for calculate the average conversion rate
unconvertedcolumn = sheet['27_Position-Valuation-MarketExposureQC']
convertedcolumn = sheet['28_Position-Valuation-MarketExposurePC']
conversionratevector = []
assetsinchf = 0
for x in rowswithchf: 
    conversionratevector.append(unconvertedcolumn.loc[x-2]/convertedcolumn.loc[x-2])
    assetsinchf = assetsinchf + convertedcolumn[x-2]     #this is going to be useful for task 2B
conversionrate = cal_average(conversionratevector)
#Now we need only multiply the sum of net assets by the average conversion rate
totalnetassetschf = totalnetassets*conversionrate

print('The value of the portofolio in CHF using the existing conversion rate is:', totalnetassetschf)

The total worth of the assets is: 100000000.0
The total number of shares: 1335776.5642484326
The value of the portofolio in CHF using the existing conversion rate is: 97421145.58230323


In [6]:
#Task 2 A
typecolumn = sheet['12_Position-InstrumentCIC']
rowswithcash = []
track = 0
totalcash = 0
for x in typecolumn:
    track = track + 1
    if x == 'XT71':
        rowswithcash.append(track+1)
for x in rowswithcash:
    totalcash = totalcash + convertedcolumn[x-2]
    
print('The total cash held is:', totalcash)
#Task 2 B
#see line 37
proportionindollars = 1- assetsinchf/totalnetassets

print('The proportion of the portofolio which is in cash assets is:' ,proportionindollars)
#Task 2 C
secondfile ='edited.xlsx'
shutil.copy("Sample TPT V3 Report.xlsx", "edited.xlsx")

edited = load_workbook(filename="edited.xlsx")
editedsheet = edited.active
# write on the edited file
for x in range(0,totalrows):
    editedsheet["J" + str(x+2)] = totalcash/totalnetassets

# Save
edited.save(filename="edited.xlsx")

In [12]:
#Task 3 A

sortedsheet =sheet.sort_values('39_Position-IntRateInst-Redemption-MaturityDate',ascending=False)
codeofsorted = sortedsheet['14_Position-InstrumentCode-Code']
first10 = []
for x in range(0,10):
    first10.append(codeofsorted[x])
print('The code of the items which will expire the latest is', first10)    


#Task 3 B
sortedsheet = sortedsheet[(pd.notnull(sortedsheet['39_Position-IntRateInst-Redemption-MaturityDate']))]
sortedsheet2=sortedsheet.assign(datenumber = lambda x: sortedsheet['39_Position-IntRateInst-Redemption-MaturityDate'] )

for i, row in sortedsheet2.iterrows():
    holder = row['39_Position-IntRateInst-Redemption-MaturityDate']
    number = int(str(holder)[:4])*1000- int(str(holder)[5:7])*100 + int(str(holder)[8:10])
    sortedsheet2.at[i,'datenumber'] = number
sortedsheet3 = sortedsheet2.sort_values('datenumber',ascending=False)

codeofsorted3 = sortedsheet3['14_Position-InstrumentCode-Code']
print('The code of the 10 items which are sorted strange is:') #different method than above
print(codeofsorted3[:10])

The code of the items which will expire the latest is ['US5323288452', 'XT8859198975', 'US7540757002', 'US4611944948', 'US9975567215', 'US1125942313', 'US9316268453', 'US9873766448', 'US7088118282', 'US9351876809']
The code of the items which are sorted strange is:
158    US7031008702
105    US1669654478
69     US9918217354
58     US9378091375
159    US7109224080
53     US1815859666
45     US6964683352
98     US6152997718
75     US1227077650
107    US6230471524
Name: 14_Position-InstrumentCode-Code, dtype: object


In [107]:
#Task 4 A

up = sortedsheet['97_Position-ContributionToSCR-MktIntUp']
down = sortedsheet['98_Position-ContributionToSCR-MktintDown']

def comquant(vector):
    quantiles = []
    value = vector.quantile(0.05)
    quantiles.append(value)
    value = vector.quantile(0.1)
    quantiles.append(value)
    value = vector.quantile(0.25)
    quantiles.append(value)
    value = vector.quantile(0.75)
    quantiles.append(value)
    value = vector.quantile(0.90)
    quantiles.append(value)
    value = vector.quantile(0.95)
    quantiles.append(value)
    return quantiles

def present(vector):
    print('The mean is:' ,stats.mean(vector))
    print('The stdev is:', stats.stdev(vector))
    print('The mode is:', mode(vector) )
    print('The 0.05, 0.1,0.25,.75,.90,.95 quantiles are, respectively:', comquant(vector))

    
present(up)
present(down)



The mean is: 29481.552414366484
The stdev is: 36860.30840808937
The mode is: ModeResult(mode=array([154.69082522]), count=array([1]))
The 0.05, 0.1,0.25,.75,.90,.95 quantiles are, respectively: [1365.02587039482, 2772.10459906838, 6272.80809086041, 32029.5150354677, 76599.73263306642, 124426.53702987099]
The mean is: -14260.880136830097
The stdev is: 19932.921748143322
The mode is: ModeResult(mode=array([-93613.59602982]), count=array([1]))
The 0.05, 0.1,0.25,.75,.90,.95 quantiles are, respectively: [-70044.13662248512, -38684.265206618635, -13784.6062804529, -2716.788862088185, -1193.1075465940398, -596.6644580467873]


In [101]:
#Task 5 

for x in range(0,totalrows):
    editedsheet["E" + str(x+2)] = totalnetassets
    editedsheet["I" + str(x+2)] = numberofshares   
    editedsheet["J" + str(x+2)] = totalcash/totalnetassets

# Save
edited.save(filename="edited.xlsx")